In [4]:
# %pip install pyyaml


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import math
import random
from dataclasses import dataclass, asdict
from datetime import datetime
from zoneinfo import ZoneInfo
import yaml

In [74]:
# load in df with all bracket info
bracket_df = pd.read_csv("C:/Users/Owner/OneDrive/Documents/cbb_survivor/cbb_survivor_sim/final_bracket_df.csv").iloc[:, 1:].copy()
bracket_df = bracket_df.loc[bracket_df["season"]==2025]

# load in df with all win prob combinations
win_prob_df = pd.read_csv("C:/Users/Owner/OneDrive/Documents/cbb_survivor/cbb_survivor_sim/final_pairs_final.csv").iloc[:, 1:].copy()
win_prob_df = win_prob_df.loc[win_prob_df["season"]==2025]

# load in df with days and round info
dates_df = pd.read_csv("C:/Users/Owner/OneDrive/Documents/cbb_survivor/cbb_survivor_sim/date_final_all.csv").iloc[:, 1:].copy()
dates_df = dates_df.loc[dates_df["season"]==2025]

In [75]:
# pull out first four teams
bracket_df_ff = bracket_df.loc[bracket_df["round"]=="First Four",["team1_name", "next_game_id1"]]
bracket_df_ff = bracket_df_ff.rename(columns = {"team1_name":"new_team", "next_game_id1":"next_game_id0"})
bracket_df_ff["join_term"] = "Winner"
ff_teams = bracket_df_ff["new_team"].unique()

# bring in potential ff winners to r1 games
bracket_join = bracket_df.copy()
bracket_join["join_term"] = bracket_join["team1_name"].str[:6]
bracket_df2 = bracket_join.merge(bracket_df_ff, on = ["next_game_id0","join_term"], how = "left")

mask = (bracket_df2["join_term"]=="Winner")
bracket_df2.loc[mask, "team1_name"] = bracket_df2.loc[mask, "new_team"]
bracket_df2 = bracket_df2.drop(["join_term","new_team"], axis=1)
bracket_df2 = bracket_df2.loc[bracket_df2["round"]=="Round of 64"]

# sort teams
bracket_df2 = bracket_df2.sort_values("team1_name")

In [76]:
# teams indices in alphabetical order
teams = bracket_df2["team1_name"].sort_values().unique()

In [77]:
# pull out first four matchup pairs of team indices
ff_indices_df = bracket_df2["team1_name"].isin(ff_teams).reset_index()
ff_indices = ff_indices_df[ff_indices_df["team1_name"]==True].index.to_list()

bracket_df_ff_short = bracket_df_ff.sort_values("new_team")
bracket_df_ff_short["id"] = ff_indices

ff_game_ids = list(bracket_df_ff_short["next_game_id0"].unique())

ff_pair_list = []
for gid in ff_game_ids:
    pair = list(bracket_df_ff_short.loc[bracket_df_ff_short["next_game_id0"]==gid, "id"].unique())
    ff_pair_list.append(pair)


In [78]:
# initialize unique game ids for each round
unique_games_r0 = bracket_df2["next_game_id0"].unique().tolist()
unique_games_r1 = bracket_df2["next_game_id1"].unique().tolist()
unique_games_r2 = bracket_df2["next_game_id2"].unique().tolist()
unique_games_r3 = bracket_df2["next_game_id3"].unique().tolist()
unique_games_r4 = bracket_df2["next_game_id4"].unique().tolist()
unique_games_r5 = bracket_df2["next_game_id5"].unique().tolist()


# map alphabetical-ordered IDs to numeric indices for matchups
unique_game_ids = unique_games_r0 + unique_games_r1 + unique_games_r2 + unique_games_r3 + unique_games_r4 + unique_games_r5
unique_game_ids.sort()

unique_game_nums = list(range(0,len(unique_game_ids)))
id_map = dict(zip(unique_game_ids, unique_game_nums))

new_bracket_df = bracket_df2.replace(id_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"])

unique_games_r0 = [id_map.get(x, x) for x in unique_games_r0]
unique_games_r1 = [id_map.get(x, x) for x in unique_games_r1]
unique_games_r2 = [id_map.get(x, x) for x in unique_games_r2]
unique_games_r3 = [id_map.get(x, x) for x in unique_games_r3]
unique_games_r4 = [id_map.get(x, x) for x in unique_games_r4]
unique_games_r5 = [id_map.get(x, x) for x in unique_games_r5]


# change unique game lists to list of lists structure
unique_games_all = [unique_games_r0, unique_games_r1, unique_games_r2, unique_games_r3, unique_games_r4, unique_games_r5]

# change team, round df to array
bracket_arr_orig = new_bracket_df.to_numpy(dtype=float)

C:\Users\Owner\AppData\Local\Temp\ipykernel_29892\2552703120.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_bracket_df = bracket_df2.replace(id_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"])


In [ ]:
# create list of round days that corresponds with unique game_id indices
dates_df_short = dates_df[["game_id", "day_num"]].sort_values("game_id")
dates_arr = dates_df_short["day_num"].to_numpy(dtype=int)

In [131]:
game_day_map = dict(zip(dates_df_short["game_id"], dates_df_short["day_num"]))

bracket_day_arr_orig = bracket_df2.replace(game_day_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"]).to_numpy()

team_day_list = []
for c in range(3):

    cur_list = bracket_day_arr_orig[:, c]

    cur_list_d1 = (cur_list == 1).astype(int)
    cur_list_d2 = (cur_list == 2).astype(int)

    cur_list_full= [cur_list_d1, cur_list_d2]


    team_day_list.append(cur_list_full)



C:\Users\Owner\AppData\Local\Temp\ipykernel_29892\1026973982.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bracket_day_arr_orig = bracket_df2.replace(game_day_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"]).to_numpy()


In [149]:
potential_opps_arr_orig = []
for r in range(6):
    cut = bracket_arr_orig[:, r]
    potential_opps_arr_cur = (cut[:, None] == cut[None, :]).astype(np.int8)
    np.fill_diagonal(potential_opps_arr_cur, 0)

    potential_opps_arr_orig.append(potential_opps_arr_cur)

In [82]:
# create a team, game_id df for which teams play in certain games
id_cols = [c for c in bracket_df2.columns if c.startswith("next_game_id")]
base_cols = ["team1_name"]

long = (
    bracket_df2[base_cols + id_cols]
      .melt(id_vars=base_cols, value_vars=id_cols, value_name="game_id")
      .dropna(subset=["game_id"])
)

team_game_matrix = (
    pd.crosstab(long["team1_name"], long["game_id"])
      .astype(int)
).sort_values("team1_name")

# change to array
game_match_arr_orig = team_game_matrix.to_numpy()


In [84]:
# create new game_id structure so we know differences between day 1 and day 2 games for simulation
unique_games_d1 = []
unique_games_d2 = []

for r in list(range(6)):

    if r <= 2:
    
        all_day1 = np.where(dates_arr==1)
        this_round = np.array(unique_games_all[r])

        mask = np.isin(this_round, all_day1)
        this_round_d1 = this_round[mask]
        this_round_d2 = this_round[~mask]

        unique_games_d1.append(this_round_d1)
        unique_games_d2.append(this_round_d2)
    
    else:
        
        this_round = np.array(unique_games_all[r])
        unique_games_d1.append(this_round)

unique_games_comb = [unique_games_d1, unique_games_d2]

In [85]:
# list for how many days per round
days_by_round = [2,2,2,1,1,1,]

In [86]:
# create array for team, team win probabilities
df = win_prob_df.drop(columns=["season"], errors="ignore").sort_values("cur_team")

idx_cols = [c for c in df.columns if c not in ["opponent", "win_prob"]]

win_prob_wide = (
    df.pivot_table(
        index=idx_cols,
        columns="opponent",
        values="win_prob",
        aggfunc="first"
    )
    .reset_index()
).sort_values("cur_team").sort_index(axis=1).drop("cur_team", axis=1)


wp_arr_orig = np.nan_to_num(win_prob_wide.to_numpy(dtype=float))

In [203]:
# initialize 1D array for teams being still in or not
still_alive_arr_orig = np.ones((68,), dtype=int)


## Break

In [204]:
still_alive_arr_orig.shape

(68,)

In [571]:
bracket_arr_orig

array([[ 9.,  3., 14.,  1., 16.,  0.],
       [13.,  5., 15.,  1., 16.,  0.],
       [38., 34., 46., 33., 16.,  0.],
       [ 6.,  2., 14.,  1., 16.,  0.],
       [ 9.,  3., 14.,  1., 16.,  0.],
       [59., 52., 62., 48., 17.,  0.],
       [38., 34., 46., 33., 16.,  0.],
       [10.,  4., 15.,  1., 16.,  0.],
       [ 7.,  2., 14.,  1., 16.,  0.],
       [45., 37., 47., 33., 16.,  0.],
       [25., 20., 31., 18., 17.,  0.],
       [55., 50., 61., 48., 17.,  0.],
       [54., 49., 61., 48., 17.,  0.],
       [39., 34., 46., 33., 16.,  0.],
       [57., 51., 62., 48., 17.,  0.],
       [ 6.,  2., 14.,  1., 16.,  0.],
       [53., 49., 61., 48., 17.,  0.],
       [24., 19., 31., 18., 17.,  0.],
       [24., 19., 31., 18., 17.,  0.],
       [56., 50., 61., 48., 17.,  0.],
       [26., 20., 31., 18., 17.,  0.],
       [23., 19., 31., 18., 17.,  0.],
       [27., 21., 32., 18., 17.,  0.],
       [43., 36., 47., 33., 16.,  0.],
       [59., 52., 62., 48., 17.,  0.],
       [28., 21., 32., 18

In [262]:
game_match_arr_orig

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(68, 62))

In [442]:
dates_arr

array([1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1])

In [221]:
wp_arr_orig

array([[0.        , 0.07466937, 0.8470308 , ..., 0.62924956, 0.21865896,
        0.41046466],
       [0.92533063, 0.        , 0.98563628, ..., 0.95461298, 0.77618701,
        0.89613846],
       [0.1529692 , 0.01436372, 0.        , ..., 0.23460269, 0.04810819,
        0.11169479],
       ...,
       [0.37075044, 0.04538702, 0.76539731, ..., 0.        , 0.14154735,
        0.29089451],
       [0.78134104, 0.22381299, 0.95189181, ..., 0.85845265, 0.        ,
        0.71329753],
       [0.58953534, 0.10386154, 0.88830521, ..., 0.70910549, 0.28670247,
        0.        ]], shape=(68, 68))

In [44]:
potential_opps_arr_orig[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(68, 68), dtype=int8)

## Break

In [148]:
potential_opps_arr_orig[2]

IndexError: list index out of range

In [236]:
@dataclass
class SimConfig:

    survivor_policies : list
    
    n_iter: int = 10

    store_tourney_results: bool = True

    run_survivor : bool = True

    wp_arr_orig = wp_arr_orig.copy()
    bracket_arr_orig = bracket_arr_orig.copy()
    game_match_arr_orig = game_match_arr_orig.copy()
    still_alive_arr_orig = still_alive_arr_orig.copy()
    potential_opps_arr_orig = potential_opps_arr_orig.copy()

    




# --------------------------------------------
# 2) Simulator object (holds state + methods)
# --------------------------------------------
class TournamentSimulator:
    def __init__(self, cfg: SimConfig, n_teams: int = 68):
        
        self.cfg = cfg

        self.n_teams = n_teams

        self.sim_id = datetime.now(ZoneInfo("America/New_York")).strftime("%Y%m%d_%H.%M.%S_%f")

        # Initialize storage for results if requested
        if self.cfg.store_tourney_results:
            self.tourney_results = np.full(
                (self.n_teams, self.cfg.n_iter),
                6,
                dtype=int
            )
        else:
            self.tourney_results = None

    def record_result(self, team_id: int, round_: int, iter_: int):
        if self.cfg.store_tourney_results:
            self.tourney_results[team_id, iter_] = round_

    
    def survivor_choose_team(self, interm_wp_arr, potential_opps_arr, r_cur, policies_alive, chosen_teams_prev):

        opps_by_nr = [1, 2, 4, 8, 16, 32]

        chosen_teams = []
        for p_iter in range(len(policies_alive)):

            if policies_alive[p_iter] == 0:

                if r_cur !=3:
                    chosen_teams.append(None)
                    
                else:
                    chosen_teams.append([None, None])
                
                continue
            
            policy = self.cfg.survivor_policies[p_iter]
            weights_list = np.array(policy["next_round_weights"][:6-r_cur])

            it=-1
            all_rd_wps = []
            for r in range(r_cur,6):

                it+=1

                final_wp_arr = potential_opps_arr[r] * interm_wp_arr

                rd_wps = final_wp_arr.sum(axis=1) / opps_by_nr[it]

                all_rd_wps.append(rd_wps)

            all_rd_adv_probs = np.cumprod(all_rd_wps, axis=1)

            obj_arr = (all_rd_adv_probs.T * weights_list).sum(axis=1)
            team_idx_order = np.argsort(obj_arr)[::-1]
            #obj_arr_sorted = obj_arr[team_idx_order]

            if r_cur != 3:
                chosen_team_cur = team_idx_order[0]
                chosen_teams.append(chosen_team_cur)
                

            else:
                chosen_teams.append([int(team_idx_order[0]), int(team_idx_order[1])])



        chosen_teams_np = np.array(chosen_teams, dtype=object)


        return chosen_teams_np





            


    def run_sim(self):

        if self.cfg.run_survivor == True:
            policies_round_out_save = []

        for i in range(self.cfg.n_iter):

            wp_arr = self.cfg.wp_arr_orig.copy()
            bracket_arr = self.cfg.bracket_arr_orig.copy()
            game_match_arr = self.cfg.game_match_arr_orig.copy()
            still_alive_arr = self.cfg.still_alive_arr_orig.copy()
            potential_opps_arr = self.cfg.potential_opps_arr_orig.copy()

            chosen_teams = None

            if self.cfg.run_survivor == True:
                policies_alive = np.ones((len(self.cfg.survivor_policies)), dtype=int)
                policies_round_out = np.full(len(self.cfg.survivor_policies), 6, dtype=int)

            # sim the first four before hand
            for g in ff_pair_list[0:4]:
                t1, t2 = g

                t1_wp = wp_arr[t1][t2]

                rand_num = random.random()

                if rand_num > t1_wp:
                    still_alive_arr[t1] = 0
                    wp_arr[t1, :] = 0
                    wp_arr[:, t1] = 0
                    game_match_arr[t1, :] = 0

                    self.record_result(t2, 0, i)
                else:
                    still_alive_arr[t2] = 0
                    wp_arr[t2, :] = 0
                    wp_arr[:, t2] = 0
                    game_match_arr[t2, :] = 0

                    self.record_result(t2, 0, i)

            # for each round
            day_cum = -1
            for r in list(range(0,6)):

                # for each day of the round
                for d in list(range(0,days_by_round[r])):

                    if r !=3:
                        day_cum += 1
                    else:
                        day_cum += 2

                    if self.cfg.run_survivor == True:
                        if r <=2:
                            interm_wp_arr = wp_arr * team_day_list[r][d][:, None]
                        else:
                            interm_wp_arr = wp_arr * 1
                        chosen_teams = self.survivor_choose_team(interm_wp_arr, potential_opps_arr, r, policies_alive, chosen_teams)

                    # for each game in the round/day
                    for g in unique_games_comb[d][r]:

                        #pull the two teams out of the game
                        t1, t2 = np.flatnonzero(game_match_arr[:, g] == 1)
                        #print([t1,t2])

                        # find the win prob for teams 1
                        t1_wp = wp_arr[t1][t2]
                        #print(t1_wp)

                        # generate number to determine winner
                        rand_num = random.random()
                        #print(rand_num)

                        # update matrices to reflect winner
                        if rand_num > t1_wp:
                            # get rid of team in still alive array
                            still_alive_arr[t1] = 0
                            wp_arr[t1, :] = 0
                            wp_arr[:, t1] = 0
                            # get rid of team's game_ids
                            game_match_arr[t1, :] = 0

                            self.record_result(t1, r+1, i)
                        else:
                            still_alive_arr[t2] = 0
                            wp_arr[t2, :] = 0
                            wp_arr[:, t2] = 0
                            game_match_arr[t2, :] = 0

                            self.record_result(t2, r+1, i)
                    
                    if self.cfg.run_survivor == True:


                        if r != 3:
                            chosen_alive = np.zeros(len(chosen_teams), dtype=int) 
                            mask = chosen_teams != None
                            idx = chosen_teams[mask].astype(int)
                            chosen_alive[mask] = still_alive_arr[idx].astype(int)

                            new_outs = (np.array(policies_alive) + np.array(chosen_alive))


                            policies_alive = np.array(policies_alive) * np.array(chosen_alive)

                            

                            policies_round_out[new_outs == 1] = day_cum
                        
                        else:
                            chosen_teams0 = chosen_teams[:,0]
                            chosen_alive0 = np.zeros(len(chosen_teams0), dtype=int) 
                            mask0 = chosen_teams0 != None
                            idx0 = chosen_teams0[mask0].astype(int)
                            chosen_alive0[mask0] = still_alive_arr[idx0].astype(int)

                            new_outs0 = (np.array(policies_alive) + np.array(chosen_alive0))
                            policies_alive = np.array(policies_alive) * np.array(chosen_alive0)


                            chosen_teams1 = chosen_teams[:,1]
                            chosen_alive1 = np.zeros(len(chosen_teams1), dtype=int) 
                            mask1 = chosen_teams1 != None
                            idx1 = chosen_teams1[mask1].astype(int)
                            chosen_alive1[mask1] = still_alive_arr[idx1].astype(int)
                            policies_alive = np.array(policies_alive) * np.array(chosen_alive1)

                            new_outs1 = (np.array(policies_alive) + np.array(chosen_alive1))


                            new_outs_all = new_outs0 + new_outs1
                            
                            policies_round_out[new_outs == 1] = day_cum
                            policies_round_out[new_outs == 2] = day_cum-1


            if self.cfg.run_survivor == True:
                policies_round_out_save.append(list(policies_round_out))

        
        if self.cfg.run_survivor == True:
            survivor_results_df = pd.DataFrame({i: col for i, col in enumerate(policies_round_out_save)})
            survivor_results_df.to_csv(f"sim_saves/survivor_results/run_{self.sim_id}_sr.csv")

                            
                            

                        

                        
                        



            

        if self.cfg.store_tourney_results == True:
            tourney_results_pdf = pd.DataFrame(self.tourney_results)
            tourney_results_pdf.to_csv(f"sim_saves/tourney_results/run_{self.sim_id}_tr.csv")

        
        cfg_dict = asdict(cfg)
        with open(f"sim_saves/params/run_{self.sim_id}_config.yaml", "w") as f:
            yaml.safe_dump(cfg_dict, f, sort_keys=False)

        

policy_list = [{"next_round_weights": [.9,.1,0,0,0,0]}]


# --------------------------------------------
# 3) Example usage
# --------------------------------------------
cfg = SimConfig(n_iter=5, store_tourney_results=True, run_survivor=True, survivor_policies=policy_list)
sim = TournamentSimulator(cfg, n_teams=68)

sim.run_sim()



In [172]:
team_day_list[3]


IndexError: list index out of range

In [603]:
tourney_results

array([[1, 1, 2, 1, 1],
       [3, 6, 4, 3, 1],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [2, 2, 1, 3, 2],
       [1, 1, 1, 1, 1],
       [2, 5, 2, 4, 6],
       [1, 1, 1, 4, 2],
       [1, 2, 1, 1, 2],
       [1, 1, 1, 1, 1],
       [3, 2, 2, 2, 2],
       [2, 2, 6, 4, 1],
       [1, 3, 3, 1, 2],
       [1, 2, 1, 2, 1],
       [1, 1, 1, 1, 1],
       [5, 4, 6, 6, 4],
       [6, 2, 2, 3, 3],
       [2, 1, 1, 1, 2],
       [1, 2, 2, 2, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [4, 5, 4, 6, 6],
       [2, 3, 2, 2, 1],
       [3, 2, 4, 2, 4],
       [2, 2, 2, 2, 4],
       [3, 1, 5, 4, 4],
       [1, 1, 1, 2, 1],
       [1, 1, 1, 1, 1],
       [4, 1, 3, 1, 2],
       [1, 2, 3, 2, 3],
       [3, 6, 2, 2, 5],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 2],
       [3, 1, 2, 1, 3],
       [6, 4, 2, 5, 2],
       [2, 3, 1, 1, 1],
       [2, 1, 2, 2, 1],
       [4, 2, 4, 3, 2],
       [1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [2, 1, 1,

In [579]:
np.flatnonzero(still_alive_arr == 1)

array([56])

In [546]:
print(f"{teams[final_teams[0]]} vs {teams[final_teams[1]]}")

Auburn vs Houston


In [578]:
teams[56]

'Texas Tech'